# Rough Bergomi vs Heston vs Black-Scholes

Final comparison — three models, one market smile.
This is what the whole project builds toward.

In [ ]:
!git clone https://github.com/adnanegrb/rough-volatility-calibrator
import os
os.chdir('rough-volatility-calibrator')

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from models.black_scholes import implied_vol
from models.heston import heston_mc
from models.rough_bergomi import simulate_rough_bergomi

sns.set_theme(style='darkgrid', palette='deep')

In [ ]:
spx = yf.Ticker('^SPX')
S   = spx.history(period='1d')['Close'].iloc[-1]

expiry = spx.options[4]
chain  = spx.option_chain(expiry)
calls  = chain.calls

calls = calls[(calls['strike'] > S * 0.85) & (calls['strike'] < S * 1.15)]
calls = calls[calls['lastPrice'] > 0.5]
calls = calls[calls['openInterest'] > 1000].reset_index(drop=True)

T = (datetime.strptime(expiry, '%Y-%m-%d') - datetime.today()).days / 365
r = 0.05

strikes       = calls['strike'].values
market_prices = calls['lastPrice'].values

market_ivols = np.array([implied_vol(p, S, K, T, r) for p, K in zip(market_prices, strikes)])
mask = ~np.isnan(market_ivols)
strikes, market_ivols = strikes[mask], market_ivols[mask]

atm_vol = market_ivols[np.argmin(np.abs(strikes - S))]

In [ ]:
# BS flat vol
bs_ivols = np.full(len(strikes), atm_vol)

# Heston smile
v0, kappa, theta, xi, rho_h = atm_vol**2, 2.0, atm_vol**2, 0.3, -0.7
heston_ivols = []
for K in strikes:
    price = heston_mc(S, K, T, r, v0, kappa, theta, xi, rho_h)
    heston_ivols.append(implied_vol(price, S, K, T, r))
heston_ivols = np.array(heston_ivols)

# Rough Bergomi — direct simulation on real spot and strikes
T_rb = max(T, 0.25)
xi0  = atm_vol**2
H, eta, rho_rb = 0.1, 1.5, -0.8

S_paths, _ = simulate_rough_bergomi(S, T_rb, r, H, eta, rho_rb,
                                     xi0=xi0, n_paths=100000, n_steps=200)
S_T = S_paths[:, -1]

rb_ivols = []
for K in strikes:
    price = np.exp(-r * T_rb) * np.mean(np.maximum(S_T - K, 0))
    iv    = implied_vol(price, S, K, T_rb, r)
    rb_ivols.append(iv if not np.isnan(iv) else 0.0)
rb_ivols = np.array(rb_ivols)

print('Done.')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(strikes, market_ivols * 100, 'o-',  linewidth=2.5, label='Market',        color=sns.color_palette()[0])
ax.plot(strikes, rb_ivols * 100,     's-',  linewidth=2.5, label='Rough Bergomi', color=sns.color_palette()[1])
ax.plot(strikes, heston_ivols * 100, '^--', linewidth=2,   label='Heston',        color=sns.color_palette()[2])
ax.plot(strikes, bs_ivols * 100,     ':',   linewidth=1.5, label='Black-Scholes', color=sns.color_palette()[3])

ax.axvline(x=S, color='gray', linestyle=':', alpha=0.5, label='ATM spot')

ax.set_xlabel('Strike', fontsize=13)
ax.set_ylabel('Implied Volatility (%)', fontsize=13)
ax.set_title(f'Volatility Smile — BS vs Heston vs Rough Bergomi  |  SPX  |  Expiry {expiry}', fontsize=14)
ax.legend(fontsize=11)

plt.tight_layout()
os.makedirs('results', exist_ok=True)
plt.savefig('results/final_comparison.png', dpi=150)

from google.colab import files
files.download('results/final_comparison.png')

plt.show()

Rough Bergomi captures the skew that both BS and Heston miss.
H ≈ 0.1 encodes the empirical roughness of realized volatility on SPX — Gatheral, Jaisson & Rosenbaum (2018).